# Lennard Jones

The Lennard-Jones system is example of a molecular dynamics simulation. 

## Initialize

Import the hoomd python package and the md component to execute MD simulations.

In [ ]:
import hoomd
import hoomd.md

Initialize the execution context to control where HOOMD will execute the simulation. When no command line options are provided, HOOMD will auto-select a GPU if it exists, or run on the CPU.

In [ ]:
hoomd.context.initialize("");

Initialize a $n$ by $n$ by $n$ simple cubic lattice of particles. The lattice initializer by default creates all particles named type "A", and with 0 velocity.

In [ ]:
hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=2.0), n=5);

## Define potential energy

$ V(r) = 4 \varepsilon \left[ \left( \frac{\sigma}{r} \right)^{12} - \left( \frac{\sigma}{r} \right)^{6} \right] $

In the Lennard-Jones system, pairs of particles closer than $r_\mathrm{cut}$ interact with this potential energy.

Choose the neighbor list acceleration structure to find neighboring particles efficiently. In systems with only one cutoff length, the cell method performs best.

In [ ]:
nl = hoomd.md.nlist.cell();

Define the functional form of the pair interaction and evaluate using the given neighbor list acceleration structure.

In [ ]:
lj = hoomd.md.pair.lj(r_cut=1.5, nlist=nl);

Specify pair potential parameters for every pair of types in the simulation.

In [ ]:
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0);

## Select integrator

The integrator defines the equations of motion that govern the system of particles, given the current configuration of the particles and the net force from all potentials. The standard integration mode in HOOMD allows different integrators to apply to different groups of particles with the same step size $dt$.

In [ ]:
hoomd.md.integrate.mode_standard(dt=0.005);

Apply the Langevin equations of motion to all particles. $kT$ defines the temperature of the system in energy units and *seed* defines the seed for the random number generator.

In [ ]:
all = hoomd.group.all();
hoomd.md.integrate.langevin(group=all, kT=0.2, seed=42);

## Write output

Periodically log the potential energy of the system to a text file.

In [ ]:
hoomd.analyze.log(filename="log-output.log",
                  quantities=['potential_energy', 'temperature'],
                  period=100,
                  overwrite=True);

Periodically write the particle configurations to a gsd file.

In [ ]:
hoomd.dump.gsd("trajectory.gsd", period=2e3, group=all, overwrite=True);

## Run the simulation

Take 10,000 steps forward in time.

In [ ]:
hoomd.run(1e4);

## Examine the output

Use matplotlib to plot the potential energy vs time step.

In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline
data = numpy.genfromtxt(fname='log-output.log', skip_header=True);

Plot the potential energy vs time step.

In [ ]:
pyplot.figure(figsize=(4,2.2), dpi=140);
#plot here

Plot temperature vs time step.

In [ ]:
pyplot.figure(figsize=(4,2.2), dpi=140);
#plot here

# Visualization

Examine how the system configuration evolves over time. [ex_render](ex_render.py) is a helper script that builds animated gifs from trajectory files and system snapshots. It is part of the [hoomd-examples](https://github.com/glotzerlab/hoomd-examples) repository and designed only to render these examples.

In [ ]:
import ex_render
ex_render.display_movie(ex_render.render_sphere_frame, 'trajectory.gsd');